In [1]:
import numpy as np
import h5py
import tensorflow as tf
import matplotlib.pyplot as plt

from keras.models import Sequential
from keras.layers import Dense 
from keras.layers import LSTM
from keras.layers import Flatten
from keras.utils import to_categorical
from keras.layers import Activation

RuntimeError: module compiled against API version 0xb but this version of numpy is 0xa

RuntimeError: module compiled against API version 0xb but this version of numpy is 0xa

Using TensorFlow backend.


In [2]:
# Creates a graph.
a = tf.constant([1.0, 2.0, 3.0, 4.0, 5.0, 6.0], shape=[2, 3], name='a')
b = tf.constant([1.0, 2.0, 3.0, 4.0, 5.0, 6.0], shape=[3, 2], name='b')
c = tf.matmul(a, b)
# Creates a session with log_device_placement set to True.
sess = tf.Session(config=tf.ConfigProto(log_device_placement=True))
# Runs the op.
print(sess.run(c))

[[ 22.  28.]
 [ 49.  64.]]


In [3]:
A01T = h5py.File('/home/carla/Downloads/project_datasets/project_datasets/A01T_slice.mat','r')
data = np.copy(A01T['image'])
labels = np.copy(A01T['type'])
labels = labels[0,0:data.shape[0]:1]
labels = np.asarray(labels, dtype=np.int32)

In [4]:
#check the data dimensionality
print data.shape
print labels.shape
data = np.swapaxes(data, 1,2)
print data.shape
print labels.shape

(288, 25, 1000)
(288,)
(288, 1000, 25)
(288,)


In [115]:
#one-hot-encode the data for use with categorical_crossentropy
labels_cata = labels - 769
print labels_cata
labels_cata = to_categorical(labels_cata, num_classes=4)
print labels_cata

[3 2 1 0 0 1 2 3 1 2 0 0 0 3 1 1 0 0 2 0 1 3 3 2 0 3 3 1 3 3 1 0 1 2 2 2 3
 2 0 3 1 2 1 2 3 1 2 0 0 0 3 1 0 2 0 2 1 3 0 2 2 0 2 1 3 3 3 2 0 3 1 3 1 0
 2 1 0 2 2 0 2 3 3 1 0 1 3 1 3 2 1 1 1 2 3 0 1 3 0 2 2 3 0 0 2 1 3 3 3 1 0
 2 1 3 0 3 2 1 3 3 0 1 1 2 3 1 0 0 3 1 0 2 1 1 2 0 3 2 2 2 2 0 1 0 1 0 0 2
 2 1 2 3 0 3 0 0 1 3 2 1 3 2 3 2 3 1 1 3 0 1 1 1 2 3 0 3 0 2 0 3 0 2 0 1 2
 2 3 0 1 3 1 2 2 0 3 1 3 0 0 2 2 1 3 1 1 0 1 3 3 1 1 1 1 3 3 2 3 0 1 2 1 0
 3 0 3 0 0 0 0 2 2 3 1 2 2 2 3 2 0 2 0 3 1 3 3 2 3 3 2 1 3 2 0 1 1 1 2 1 3
 2 3 1 2 0 3 0 2 3 0 2 0 1 1 0 3 0 3 2 2 0 2 1 1 0 2 0 1 0]
[[ 0.  0.  0.  1.]
 [ 0.  0.  1.  0.]
 [ 0.  1.  0.  0.]
 ..., 
 [ 1.  0.  0.  0.]
 [ 0.  1.  0.  0.]
 [ 1.  0.  0.  0.]]


In [113]:
#vanilla LSTM implementation
model = Sequential()
model.add(LSTM(5, input_shape=(1000, 25), ))
model.add(Dense(4, activation= 'softmax'))
model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])
model.fit(data, labels_cata, nb_epoch=2, batch_size=8, verbose=1, validation_split=0.15)
print(model.summary())

Train on 244 samples, validate on 44 samples
Epoch 1/2
244/244 [==============================] - 35s 145ms/step - loss: nan - acc: 0.2131 - val_loss: nan - val_acc: 0.2500
Epoch 2/2
244/244 [==============================] - 25s 102ms/step - loss: nan - acc: 0.2500 - val_loss: nan - val_acc: 0.2500
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_103 (LSTM)              (None, 5)                 620       
_________________________________________________________________
dense_62 (Dense)             (None, 4)                 24        
Total params: 644
Trainable params: 644
Non-trainable params: 0
_________________________________________________________________
None


In [111]:
model = Sequential()
model.add(LSTM(10, input_shape=(1000,25),return_sequences=True))
model.add(LSTM(10, return_sequences=True))
model.add(LSTM(10))
model.add(Dense(4))
model.add(Activation('softmax'))
model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])
model.fit(data, labels_cata, nb_epoch=3, batch_size=10, verbose=1, validation_split=0.1)
print(model.summary())

Train on 259 samples, validate on 29 samples
Epoch 1/3
259/259 [==============================] - 74s 284ms/step - loss: nan - acc: 0.2896 - val_loss: nan - val_acc: 0.3448
Epoch 2/3
259/259 [==============================] - 64s 247ms/step - loss: nan - acc: 0.2394 - val_loss: nan - val_acc: 0.3448
Epoch 3/3
259/259 [==============================] - 61s 237ms/step - loss: nan - acc: 0.2394 - val_loss: nan - val_acc: 0.3448


In [99]:
# Reference model
from keras.models import Sequential
from keras.layers import LSTM, Dense
import numpy as np

data_dim = 16
timesteps = 8
num_classes = 10

# expected input data shape: (batch_size, timesteps, data_dim)
model = Sequential()
model.add(LSTM(32, return_sequences=True,
               input_shape=(timesteps, data_dim)))  # returns a sequence of vectors of dimension 32
model.add(LSTM(32, return_sequences=True))  # returns a sequence of vectors of dimension 32
model.add(LSTM(32))  # return a single vector of dimension 32
model.add(Dense(10, activation='softmax'))

model.compile(loss='categorical_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])

# Generate dummy training data
x_train = np.random.random((1000, timesteps, data_dim))
y_train = np.random.random((1000, num_classes))

# Generate dummy validation data
x_val = np.random.random((100, timesteps, data_dim))
y_val = np.random.random((100, num_classes))

model.fit(x_train, y_train,
          batch_size=64, epochs=10,
          validation_data=(x_val, y_val))

Train on 1000 samples, validate on 100 samples
Epoch 1/10
1000/1000 [==============================] - 10s 10ms/step - loss: 11.6029 - acc: 0.1000 - val_loss: 11.4855 - val_acc: 0.0600
Epoch 2/10
1000/1000 [==============================] - 0s 373us/step - loss: 11.6008 - acc: 0.0960 - val_loss: 11.4851 - val_acc: 0.0900
Epoch 3/10
1000/1000 [==============================] - 0s 372us/step - loss: 11.6005 - acc: 0.0900 - val_loss: 11.4878 - val_acc: 0.0900
Epoch 4/10
1000/1000 [==============================] - 0s 401us/step - loss: 11.6001 - acc: 0.1080 - val_loss: 11.4819 - val_acc: 0.0900
Epoch 5/10
1000/1000 [==============================] - 0s 396us/step - loss: 11.5997 - acc: 0.1140 - val_loss: 11.4861 - val_acc: 0.0900
Epoch 6/10
1000/1000 [==============================] - 0s 401us/step - loss: 11.5991 - acc: 0.1240 - val_loss: 11.4835 - val_acc: 0.0900
Epoch 7/10
1000/1000 [==============================] - 0s 401us/step - loss: 11.5991 - acc: 0.1240 - val_loss: 11.4870 - val

In [62]:
#Check the shape of the model and the inputs
model.summary()
print "Inputs: {}".format(model.input_shape)
print "Outputs: {}".format(model.output_shape)
print "Actual input: {}".format(x_train.shape)
print "Actual output: {}".format(y_train.shape)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_42 (LSTM)               (None, 8, 32)             6272      
_________________________________________________________________
lstm_43 (LSTM)               (None, 8, 32)             8320      
_________________________________________________________________
lstm_44 (LSTM)               (None, 32)                8320      
_________________________________________________________________
dense_27 (Dense)             (None, 10)                330       
Total params: 23,242
Trainable params: 23,242
Non-trainable params: 0
_________________________________________________________________
Inputs: (None, 8, 16)
Outputs: (None, 10)
Actual input: (1000, 8, 16)
Actual output: (1000, 10)
